In [1]:
import twitter


# initialize api instance

twitter_api = twitter.Api(consumer_key = 'W6XQbc8fzjsTGBOvTbtfVxGfj',
                         consumer_secret='hIurrqdfIbGgmftvCxQvolsjZDWKhvCmD1P4Qf2yShpUbDKYjT',
                         access_token_key = '1233241924716433408-uPYHdzc6jkGGuB316D5865BmO9LaWY',
                         access_token_secret = 'XGbRRNBqMq3e4moXiRKgUke3fCcHMYtccAVDuKV7OObCD')

# test authentication
print(twitter_api.VerifyCredentials())

{"created_at": "Fri Feb 28 04:06:19 +0000 2020", "default_profile": true, "default_profile_image": true, "friends_count": 10, "id": 1233241924716433408, "id_str": "1233241924716433408", "name": "Shreshtha Kanungo", "profile_background_color": "F5F8FA", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_image_url_https": "https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "ShreshthaKanun1"}


In [2]:
def buildTestSet(search_keyword):
    try:
        tweets_fetched = twitter_api.GetSearch(search_keyword, count = 100)
        
        print("Fetched " + str(len(tweets_fetched)) + " tweets for the term " + search_keyword)
        
        return [{"text":status.text, "label":None} for status in tweets_fetched]
    except:
        print("Unfortunately, something went wrong..")
        return None

In [3]:
search_term = input("Enter a search keyword:")
testDataSet = buildTestSet(search_term)

print(testDataSet[0:4])

Enter a search keyword:Corn
Fetched 100 tweets for the term Corn
[{'text': "*Fergalicious comes on radio*\n\ndaughter, 12: I like this song, who sings it?\n\nme: Fergie!\n\ndaughter: Who's Fergie?… https://t.co/teyhWuCxLV", 'label': None}, {'text': 'People should take as long as they need. But if you think that Cory was sitting around deciding his vote, rather th… https://t.co/J2h8zvtQ2O', 'label': None}, {'text': 'Eating beef isn’t just bad for the climate. The irrigation of alfalfa, hay, maize (corn), sorghum and other crops e… https://t.co/94r2Nz5e4T', 'label': None}, {'text': 'RT @stayfrea_: I Refuse To Let The Convo Die If I Like You, I Start Asking Mad Shit. You Like Corn?', 'label': None}]


In [4]:
def buildTrainingSet(corpusFile, tweetDataFile):
    import csv
    import time
    
    corpus = []
    trainingDataset = []
    
    with open(corpusFile,'r') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',', quotechar = "\"")
        for row in lineReader:
            corpus.append({"tweet_id":row[2], "label":row[1], "topic":row[0]})
            break
            
    rate_limit=180
    sleep_time = 900/180
    
    #trainingDataset = []
    
    for tweet in corpus:
        try:
            status = twitter_api.GetStatus(tweet["tweet_id"])
            print("Tweet Fetched"+status.text)
            tweet["text"] = status.text
            trainingDataset.append(tweet)
            time.sleep(sleep_time)
        except:
            continue
            
            
    # now we write them to the empty CSV file
    with open(tweetDataFile,'w') as csvfile:
        linewriter = csv.writer(csvfile,delimiter=',',quotechar="\"")
        for tweet in trainingDataset:
            try:
                linewriter.writerow([tweet["tweet_id"], tweet["text"], tweet["label"], tweet["topic"]])
            except Exception as e:
                print(e)
    return trainingDataset

In [7]:
corpusFile = "/Users/Ashwin/Desktop/twitter-sentiment-training-master/corpus.csv"
tweetDataFile = "/Users/Ashwin/Desktop/twitter-sentiment-training-master/tweetDataFile.csv"

trainingData = buildTrainingSet(corpusFile, tweetDataFile)

Tweet FetchedNow all @Apple has to do is get swype on the iphone and it will be crack. Iphone that is


# Pre-Processing

In [8]:
import re
import nltk
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords
nltk.download('punkt')

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]

[nltk_data] Downloading package punkt to /Users/Ashwin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
tweetProcessor = PreProcessTweets()
preprocessedTrainingSet = tweetProcessor.processTweets(trainingData)
preprocessedTestSet = tweetProcessor.processTweets(testDataSet)

# Building the vocabulary

In [14]:

def buildVocabulary(preprocessedTrainingSet):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingSet:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

In [15]:
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

In [17]:

word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)

In [18]:

NBayesClassifier = nltk.NaiveBayesClassifier.train(trainingFeatures)

In [20]:
NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

# get the majority vote
if NBResultLabels.count('positive') > NBResultLabels.count('negative'):
    print("Overall Positive Sentiment")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
else: 
    print("Overall Negative Sentiment")
    print("Negative Sentiment Percentage = " + str(100*NBResultLabels.count('negative')/len(NBResultLabels)) + "%")

Overall Positive Sentiment
Positive Sentiment Percentage = 100.0%
